In [ ]:
import pandas as pd
import pickle
from collections import Counter
from math import sqrt
from tqdm import tqdm
pro_df = pd.read_csv("../data/products_train.csv", sep=',')
product_asin2id = pickle.load(open('../data/product_asin2id.pkl', 'rb'))
locale2ids = pickle.load(open('../data/local2ids.pkl', 'rb'))
item = pro_df['id']
item = [product_asin2id[x] for x in item]
pro_df['item'] = item

In [ ]:
"""
When generating the training and validation sets,
you need to select 'train', 
and when generating the test set, 
you need to select 'test'.
When this parameter is modified, your matrix needs to be rebuilt.
"""

# data_type = 'test'
data_type = 'train'

In [ ]:
def c_plus(counter, number):
    ans = Counter()
    while number!=0:
        if number%2==1:
            ans += counter
        number = number // 2
        if number!=0:
            counter = counter+counter
    return ans

train_data = pd.read_pickle('../data/train_data_005.dataset').reset_index(drop=True)
valid_data = pd.read_pickle('../data/valid_data_005.dataset').reset_index(drop=True)
test_data = pd.read_pickle('../data/test_data_p2.dataset').reset_index(drop=True)
test_data2 = pd.read_pickle('../data/test_data.dataset').reset_index(drop=True)
test_data3 = pd.read_pickle('../data/test_data_p3.dataset').reset_index(drop=True)
test_data2 = pd.concat([test_data2,test_data3]).reset_index(drop=True)
train_data['user'] = range(len(train_data))
train_data['user'] = train_data['user'].astype('int32')
valid_data['user'] = range(len(train_data),len(train_data)+len(valid_data))
valid_data['user'] = valid_data['user'].astype('int32')
test_data['user'] = range(len(train_data)+len(valid_data),len(train_data)+len(valid_data)+len(test_data))
test_data['user'] = test_data['user'].astype('int32')
train_session = train_data['session'].tolist()
train_locale = train_data['locale'].tolist()
train_user = train_data['user'].tolist()
valid_session = valid_data['session'].tolist()
valid_locale = valid_data['locale'].tolist()
valid_user = valid_data['user'].tolist()
test_session = test_data['session'].tolist()
test_locale = test_data['locale'].tolist()
test_user = test_data['user'].tolist()
test_session2 = test_data2['session'].tolist()
test_locale2 = test_data2['locale'].tolist()

### string rem

In [ ]:
check1 = {}
counter1 = Counter()
train_string1 = []
for i in tqdm(range(len(train_session))):
    string = ""
    row = train_session[i]
    pre = row[:-1]
    for item in pre[::-1]:
        string = string + str(item)+' '
        if counter1[string] == 0:
            counter1[string] +=1
            check1[string] = Counter()
        else:
            counter1[string] +=1
        check1[string][row[-1]] += 1
    train_string1.append(string)
train_data['string1'] = train_string1

valid_string1 = []
for i in tqdm(range(len(valid_session))):
    string = ""
    row = valid_session[i]
    pre = row[:-1]
    for item in pre[::-1]:
        string = string + str(item)+' '
        if counter1[string] == 0:
            counter1[string] +=1
            check1[string] = Counter()
        else:
            counter1[string] +=1
        check1[string][row[-1]] += 1
    valid_string1.append(string)
valid_data['string1'] = valid_string1
    
test_string1 = []
for i in tqdm(range(len(test_session))):
    string = ""
    row = test_session[i]
    pre = row
    for item in pre[::-1]:
        string = string + str(item)+' '
    test_string1.append(string)
test_data['string1'] = test_string1

check3 = {}
counter3 = Counter()
train_string3 = []
for i in tqdm(range(len(train_session))):
    string = ""
    row = train_session[i]
    pre = row[:-1]
    pre = list(dict.fromkeys(pre[::-1]))[::-1]
    for item in pre[::-1]:
        string = string + str(item)+' '
        if counter3[string] == 0:
            counter3[string] +=1
            check3[string] = Counter()
        else:
            counter3[string] +=1
        check3[string][row[-1]] += 1
    train_string3.append(string)
train_data['string3'] = train_string3
valid_string3 = []
for i in tqdm(range(len(valid_session))):
    string = ""
    row = valid_session[i]
    pre = row[:-1]
    pre = list(dict.fromkeys(pre[::-1]))[::-1]
    for item in pre[::-1]:
        string = string + str(item)+' '
        if counter3[string] == 0:
            counter3[string] +=1
            check3[string] = Counter()
        else:
            counter3[string] +=1
        check3[string][row[-1]] += 1
    valid_string3.append(string)
valid_data['string3'] = valid_string3

test_string3 = []
for i in tqdm(range(len(test_session))):
    string = ""
    row = test_session[i]
    pre = row
    pre = list(dict.fromkeys(pre[::-1]))[::-1]
    for item in pre[::-1]:
        string = string + str(item)+' '
    test_string3.append(string)
test_data['string3'] = test_string3

100%|██████████| 316972/316972 [00:00<00:00, 319655.49it/s]


### process

In [ ]:
for i in tqdm(range(len(train_session))):
    row = []
    for j in range(len(train_session[i])):
        if j != 0:
            if train_session[i][j] == train_session[i][j-1]:
                continue
        row.append(train_session[i][j])
    train_session[i] = row
train_data['session'] = train_session
    
for i in tqdm(range(len(valid_session))):
    row = []
    for j in range(len(valid_session[i])):
        if j != 0:
            if valid_session[i][j] == valid_session[i][j-1]:
                continue
        row.append(valid_session[i][j])
    valid_session[i] = row
valid_data['session'] = valid_session

for i in tqdm(range(len(test_session))):
    row = []
    for j in range(len(test_session[i])):
        if j != 0:
            if test_session[i][j] == test_session[i][j-1]:
                continue
        row.append(test_session[i][j])
    test_session[i] = row
test_data['session'] = test_session

for i in tqdm(range(len(test_session2))):
    row = []
    for j in range(len(test_session2[i])):
        if j != 0:
            if test_session2[i][j] == test_session2[i][j-1]:
                continue
        row.append(test_session2[i][j])
    test_session2[i] = row
test_data2['session'] = test_session2

100%|██████████| 376971/376971 [00:01<00:00, 264053.02it/s]


# init feature

In [ ]:
import os
import pickle
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm


In [ ]:
itemnum = 1410675 + 1
#locale map number
locale_map = {}
temp = 0
for item in list(set(train_locale)):
    locale_map[item] = temp
    temp += 1

In [ ]:
def get_max(cov,number):
    max_ = [[ 0 for _ in range(number)] for i in range(3)]
    for i in range(3):
        for j in range(0,number):
            max_[i][j] = sum(cov[i][j].values())
    return max_

### price feature

In [ ]:
price = pro_df[pro_df['locale']=='DE']['price'].tolist()
for i in range(len(price)):
    if price[i] == 40000000.07:
        price[i] = -1
print('DE',max(price))
price = pro_df[pro_df['locale']=='JP']['price'].tolist()
for i in range(len(price)):
    if price[i] == 40000000.07:
        price[i] = -1
print('JP',max(price))
price = pro_df[pro_df['locale']=='UK']['price'].tolist()
for i in range(len(price)):
    if price[i] == 40000000.07:
        price[i] = -1
print('UK',max(price))

DE 9180.0
JP 3933800.0
UK 7999.0


In [ ]:
from tqdm import tqdm
for locale_ in ['DE','JP','UK']:
    price = []
    item = []
    temp = 1.
    max_price = 9180
    min_price = 0.01
    if locale_ == 'JP':
        max_price = 3933800
        min_price = 1
    elif locale_=='UK':
        min_price = 0.01
        max_price = 7999

    temp_df = pro_df[pro_df['locale']==locale_]
    temp_price = temp_df['price'].tolist()
    temp_item = temp_df['item'].tolist()
    for i in tqdm(range(len(temp_price))):
        item.append(temp_item[i])
        if temp_price[i] == 40000000.07:
            price.append(1)
        elif temp_price[i] == 0:
            price.append(min_price/max_price)
        else:
            price.append(temp_price[i]/max_price)
    price_df = pd.DataFrame({'item':item,'price':price}).set_index(["item"])
    price_df['price'] = price_df['price'].astype('float32')
    price_df.to_parquet('item_price'+locale_+'.pqt')

100%|██████████| 500180/500180 [00:00<00:00, 1436405.09it/s]


In [ ]:
for locale_ in ['DE','JP','UK']:
    new_user = []
    price_power_mean = []
    price_power_1_mean = []
    price_power_2_mean = []
    price_power_3_mean = []
    price_power_5_mean = []
    
    price_df = pd.read_parquet('item_price'+locale_+'.pqt')
    price_dict = dict(zip(price_df.index.tolist(), price_df['price'].tolist()))

    for i in tqdm(range(len(train_session))):
        if train_locale[i] != locale_:
            continue
        new_user.append(train_user[i])
        row = train_session[i][:-1]
        row = list(dict.fromkeys(row[::-1]))
        scores = 0.
        scores1 = 0.
        scores2 = 0.
        scores3 = 0.
        scores5 = 0.
        for j in row:
            scores +=  price_dict[j]
        for j in row[:1]:
            scores1 +=  price_dict[j]
        for j in row[:2]:
            scores2 +=  price_dict[j]
        for j in row[:3]:
            scores3 +=  price_dict[j]
        for j in row[:5]:
            scores5 +=  price_dict[j]
        scores/=len(row)
        scores1/=len(row[:1])
        scores2/=len(row[:2])
        scores3/=len(row[:3])
        scores5/=len(row[:5])
        price_power_mean.append(scores)
        price_power_1_mean.append(scores1)
        price_power_2_mean.append(scores2)
        price_power_3_mean.append(scores3)
        price_power_5_mean.append(scores5)
        
    for i in tqdm(range(len(valid_session))):
        if valid_locale[i] != locale_:
            continue
        new_user.append(valid_user[i])
        row = valid_session[i][:-1]
        row = list(dict.fromkeys(row[::-1]))
        scores = 0.
        scores = 0.
        scores1 = 0.
        scores2 = 0.
        scores3 = 0.
        scores5 = 0.
        for j in row:
            scores +=  price_dict[j]
        for j in row[:1]:
            scores1 +=  price_dict[j]
        for j in row[:2]:
            scores2 +=  price_dict[j]
        for j in row[:3]:
            scores3 +=  price_dict[j]
        for j in row[:5]:
            scores5 +=  price_dict[j]
        scores/=len(row)
        scores1/=len(row[:1])
        scores2/=len(row[:2])
        scores3/=len(row[:3])
        scores5/=len(row[:5])
        price_power_mean.append(scores)
        price_power_1_mean.append(scores1)
        price_power_2_mean.append(scores2)
        price_power_3_mean.append(scores3)
        price_power_5_mean.append(scores5)
        
    for i in tqdm(range(len(test_session))):
        if test_locale[i] != locale_:
            continue
        new_user.append(test_user[i])
        row = test_session[i]
        row = list(dict.fromkeys(row[::-1]))
        scores = 0.
        scores1 = 0.
        scores2 = 0.
        scores3 = 0.
        scores5 = 0.
        for j in row:
            scores +=  price_dict[j]
        for j in row[:1]:
            scores1 +=  price_dict[j]
        for j in row[:2]:
            scores2 +=  price_dict[j]
        for j in row[:3]:
            scores3 +=  price_dict[j]
        for j in row[:5]:
            scores5 +=  price_dict[j]
        scores/=len(row)
        scores1/=len(row[:1])
        scores2/=len(row[:2])
        scores3/=len(row[:3])
        scores5/=len(row[:5])
        price_power_mean.append(scores)
        price_power_1_mean.append(scores1)
        price_power_2_mean.append(scores2)
        price_power_3_mean.append(scores3)
        price_power_5_mean.append(scores5)
        
    user_price = pd.DataFrame({'user':new_user,
                               'price_power_0_mean':price_power_mean,
                               'price_power_1_mean':price_power_1_mean,
                               'price_power_2_mean':price_power_2_mean,
                               'price_power_3_mean':price_power_3_mean,
                               'price_power_5_mean':price_power_5_mean,
                              }).set_index(["user"])
    user_price['price_power_0_mean'] = user_price['price_power_0_mean'].astype('float32')
    user_price['price_power_1_mean'] = user_price['price_power_1_mean'].astype('float32')
    user_price['price_power_2_mean'] = user_price['price_power_2_mean'].astype('float32')
    user_price['price_power_3_mean'] = user_price['price_power_3_mean'].astype('float32')
    user_price['price_power_5_mean'] = user_price['price_power_5_mean'].astype('float32')
    user_price.to_parquet('user_price'+locale_+'.pqt')

100%|██████████| 316972/316972 [00:00<00:00, 349824.88it/s] 


## matrix feature

### 1.next item

In [ ]:
def make_item_next(cov,row,locale__,apper,_max=None):
    for j in range(len(row)-1):
        cov[locale__][row[j]][row[j+1]] += apper
        if _max!=None:
            _max[locale__][row[j]] += apper

In [ ]:
item_next = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_item_next(item_next,row,locale__,1)
        
for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_item_next(item_next,row,locale__,1)
    
if data_type == 'test':
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        locale__ = locale_map[test_locale[i]]
        make_item_next(item_next,row,locale__,3)

    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        locale__ = locale_map[test_locale2[i]]
        make_item_next(item_next,row,locale__,2)

100%|██████████| 376971/376971 [00:01<00:00, 306261.07it/s]


In [ ]:
item_next_max = get_max(item_next,itemnum)

### 2.next item-skip2

In [ ]:
def make_item_next2(cov,row,locale__,apper,_max=None): 
    for j in range(len(row)-2):
        cov[locale__][row[j]][row[j+2]] += apper
        if _max!=None:
            _max[locale__][row[j]] += apper

In [ ]:
item_next2 = [[ Counter() for _ in range(itemnum)] for i in range(temp)]

for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_item_next2(item_next2,row,locale__,1)
        
for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_item_next2(item_next2,row,locale__,1)
    
if data_type == 'test':
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        locale__ = locale_map[test_locale[i]]
        make_item_next2(item_next2,row,locale__,2)
        
    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        locale__ = locale_map[test_locale2[i]]
        make_item_next2(item_next2,row,locale__,2)

100%|██████████| 376971/376971 [00:01<00:00, 337179.90it/s]


In [ ]:
item_next2_max = get_max(item_next2,itemnum)

### 3.next item-skip3

In [ ]:
def make_item_next3(cov,row,locale__,apper,_max=None): 
    for j in range(len(row)-3):
        cov[locale__][row[j]][row[j+3]] += apper
        if _max!=None:
            _max[locale__][row[j]] += apper

In [ ]:
item_next3 = [[ Counter() for _ in range(itemnum)] for i in range(temp)]

for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_item_next3(item_next3,row,locale__,1)
        
for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_item_next3(item_next3,row,locale__,1)
    
if data_type == 'test':
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        locale__ = locale_map[test_locale[i]]
        make_item_next3(item_next3,row,locale__,2)
    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        locale__ = locale_map[test_locale2[i]]
        make_item_next3(item_next3,row,locale__,2)

100%|██████████| 376971/376971 [00:00<00:00, 447324.85it/s]


In [ ]:
item_next3_max = get_max(item_next3,itemnum)

### 4.back item

In [ ]:
def make_item_next_back(cov,row,locale__,apper,_max=None):
    for j in range(len(row)-1):
        cov[locale__][row[j+1]][row[j]] += apper
        if _max!=None:
            _max[locale__][row[j+1]] += apper

In [ ]:
item_next_back = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_item_next_back(item_next_back,row,locale__,1)
    
for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_item_next_back(item_next_back,row,locale__,1)
if data_type == 'test':
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        locale__ = locale_map[test_locale[i]]
        make_item_next_back(item_next_back,row,locale__,2)
    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        locale__ = locale_map[test_locale2[i]]
        make_item_next_back(item_next_back,row,locale__,2)

100%|██████████| 16364/16364 [00:00<00:00, 192166.66it/s]


In [ ]:
item_next_back_max = get_max(item_next_back,itemnum)

### 5.back item-skip2

In [ ]:
def make_item_next2_back(cov,row,locale__,apper,_max=None): 
    for j in range(len(row)-2):
        cov[locale__][row[j+2]][row[j]] += apper
        if _max!=None:
            _max[locale__][row[j+2]] += apper

In [ ]:
item_next2_back = [[ Counter() for _ in range(itemnum)] for i in range(temp)]

for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_item_next2_back(item_next2_back,row,locale__,1)
for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_item_next2_back(item_next2_back,row,locale__,1)
if data_type == 'test':
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        locale__ = locale_map[test_locale[i]]
        make_item_next2_back(item_next2_back,row,locale__,2)
    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        locale__ = locale_map[test_locale2[i]]
        make_item_next2_back(item_next2_back,row,locale__,2)

100%|██████████| 376971/376971 [00:01<00:00, 316030.89it/s]


In [ ]:
item_next2_back_max = get_max(item_next2_back,itemnum)

### 6.last item

In [ ]:
def make_item_next_last(cov,row,locale__,apper,_max=None): 
    for j in range(len(row)-1):
        cov[locale__][row[j]][row[-1]] += apper
        if _max!=None:
            _max[locale__][row[j]] += apper

In [ ]:
item_next_last_0_2 = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
item_next_last_2_3 = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
# item_next_last_3_5 = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
item_next_last_5_5 = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_item_next_last(item_next_last_0_2,row[-2:],locale__,1)
    make_item_next_last(item_next_last_2_3,row[-3:-2] + [row[-1]],locale__,1)
    # make_item_next_last(item_next_last_3_5,row[-5:-3]+ [row[-1]],locale__,1)
    make_item_next_last(item_next_last_5_5,row[:-3]+ [row[-1]],locale__,1)
for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_item_next_last(item_next_last_0_2,row[-2:],locale__,1)
    make_item_next_last(item_next_last_2_3,row[-3:-2] + [row[-1]],locale__,1)
    # make_item_next_last(item_next_last_3_5,row[-5:-3]+ [row[-1]],locale__,1)
    make_item_next_last(item_next_last_5_5,row[:-3]+ [row[-1]],locale__,1)
    

100%|██████████| 16364/16364 [00:00<00:00, 143843.69it/s]


In [ ]:
item_next_last_0_2_max = get_max(item_next_last_0_2,itemnum)
item_next_last_2_3_max = get_max(item_next_last_2_3,itemnum)
# item_next_last_3_5_max = get_max(item_next_last_3_5,itemnum)
item_next_last_5_5_max = get_max(item_next_last_5_5,itemnum)

### 7.time last item

In [ ]:
#train data

item_last_time = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
def make_item_last_time(cov,row,locale__,apper,_max=None):
    weight = 0.
    temp_list = [1,2,3,5,20]
    for j in range(2,len(row)+1):
        if j >10:
            break
        for k in range(len(temp_list)):
            if j-1<= temp_list[k]:
                weight = 1.0/pow(20,k)
                break
        cov[locale__][row[-j]][row[-1]] += apper*weight
        if _max!=None:
            _max[locale__][row[-j]] += apper*weight
for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_item_last_time(item_last_time,row,locale__,1)
            
for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_item_last_time(item_last_time,row,locale__,1)   

100%|██████████| 16364/16364 [00:00<00:00, 117590.93it/s]


In [ ]:
item_last_time_max = get_max(item_last_time,itemnum)

### 8.cf model

In [ ]:
import math
item_sim = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
item_cnt = [[ 0 for _ in range(itemnum)] for i in range(temp)]
for i in tqdm(range(len(train_session))):
    if i%10==0 and data_type=='train':
        row = train_session[i][:-1]
    else:
        row = train_session[i]
    locale_ = locale_map[train_locale[i]]
    for j in range(len(row)):
        item_cnt[locale_][row[j]]+=1
        # right = list(dict.fromkeys(row[j:]))
        # left = list(dict.fromkeys(row[:j+1:-1]))
        right = row[j:]
        left = row[:j+1:-1]
        le = len(right)+len(left) - 2
        if le == 0:
            continue
        for k in range(1,len(right)):
            loc_weight = 0.8**(k-1)
            item_sim[locale_][row[j]][right[k]] += loc_weight/math.log(1 + le)
        for k in range(1,len(left)):
            loc_weight = 0.7**(k-1)
            item_sim[locale_][row[j]][left[k]] += loc_weight/math.log(1 + le)
for i in tqdm(range(len(valid_session))):
    row = valid_session[i][:-1]
    locale_ = locale_map[valid_locale[i]]
    for j in range(len(row)):
        item_cnt[locale_][row[j]]+=1
        # right = list(dict.fromkeys(row[j:]))
        # left = list(dict.fromkeys(row[:j+1:-1]))
        right = row[j:]
        left = row[:j+1:-1]
        le = len(right)+len(left) - 2
        if le == 0:
            continue
        for k in range(1,len(right)):
            loc_weight = 0.8**(k-1)
            item_sim[locale_][row[j]][right[k]] += loc_weight/math.log(1 + le)
        for k in range(1,len(left)):
            loc_weight = 0.7**(k-1)
            item_sim[locale_][row[j]][left[k]] += loc_weight/math.log(1 + le)
if data_type == 'test':
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        locale_ = locale_map[test_locale[i]]
        for j in range(len(row)):
            item_cnt[locale_][row[j]]+=1
            # right = list(dict.fromkeys(row[j:]))
            # left = list(dict.fromkeys(row[:j+1:-1]))
            right = row[j:]
            left = row[:j+1:-1]
            le = len(right)+len(left) - 2
            if le == 0:
                continue
            for k in range(1,len(right)):
                loc_weight = 0.8**(k-1)
                item_sim[locale_][row[j]][right[k]] += loc_weight/math.log(1 + le)
            for k in range(1,len(left)):
                loc_weight = 0.7**(k-1)
                item_sim[locale_][row[j]][left[k]] += loc_weight/math.log(1 + le)
    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        locale_ = locale_map[test_locale2[i]]
        for j in range(len(row)):
            item_cnt[locale_][row[j]]+=1
            # right = list(dict.fromkeys(row[j:]))
            # left = list(dict.fromkeys(row[:j+1:-1]))
            right = row[j:]
            left = row[:j+1:-1]
            le = len(right)+len(left) - 2
            if le == 0:
                continue
            for k in range(1,len(right)):
                loc_weight = 0.8**(k-1)
                item_sim[locale_][row[j]][right[k]] += loc_weight/math.log(1 + le)
            for k in range(1,len(left)):
                loc_weight = 0.7**(k-1)
                item_sim[locale_][row[j]][left[k]] += loc_weight/math.log(1 + le)
for locale_ in range(temp):
    for i in tqdm(range(itemnum)):
        for j in item_sim[locale_][i].keys():
            item_sim[locale_][i][j] = item_sim[locale_][i][j]/ math.sqrt(item_cnt[locale_][i] * item_cnt[locale_][j])

100%|██████████| 1410676/1410676 [00:09<00:00, 146360.18it/s] 


### 9.unique_pair

In [ ]:
#train data
unique_pair = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
def make_unique_pair(cov,row,locale__,apper,_max=None):
    row = list(set(row))
    for j in range(len(row)):
        for k in range(j+1,len(row)):
            cov[locale__][row[j]][row[k]] += apper
            cov[locale__][row[k]][row[j]] += apper
            if _max !=None:
                _max[locale__][row[j]]+=apper
                _max[locale__][row[k]]+=apper
apper = 1
for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_unique_pair(unique_pair,row,locale__,1)

for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_unique_pair(unique_pair,row,locale__,1)
if data_type == 'test':
    apper = 2
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        locale__ = locale_map[test_locale[i]]
        make_unique_pair(unique_pair,row,locale__,2)
        
    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        locale__ = locale_map[test_locale2[i]]
        make_unique_pair(unique_pair,row,locale__,2)

100%|██████████| 376971/376971 [00:04<00:00, 77471.66it/s] 


In [ ]:
unique_pair_max = get_max(unique_pair,itemnum)

### 10.brand

In [ ]:
apper = 1
brand = pro_df['brand'].tolist()
brand = [str(x) for x in brand]
brand_key = list(set(brand))
brand_number = len(brand_key)
brand_values = list(range(1,len(brand_key)+1))
brand_s2n = dict(zip(brand_key, brand_values))
brand_s2n['nan'] = 0
brand_map = [{} for i in range(temp)]
brand_locale = pro_df['locale'].tolist()
brand_item = pro_df['item'].tolist()

for i in range(len(brand)):
    if brand_locale[i] not in ['DE','UK','JP']:
        continue
    brand_map[locale_map[brand_locale[i]]][brand_item[i]]= brand_s2n[brand[i]]

brand_window = [[ Counter() for _ in range(brand_number+2)] for i in range(temp)]
def make_brand_window(cov,row,locale__,apper,max_=None):
    for j in range(len(row)-1):
        a = brand_map[locale__][row[j]]
        b = brand_map[locale__][row[j+1]]
        if a*b==0:
            continue
        cov[locale__][a][b] += apper
        cov[locale__][b][a] += apper
        if max_!=None:
            max_[locale__][a] += apper
            max_[locale__][b] += apper
    for j in range(len(row)-2):
        a = brand_map[locale__][row[j]]
        b = brand_map[locale__][row[j+2]]
        if a*b==0:
            continue
        cov[locale__][a][b] += apper
        cov[locale__][b][a] += apper
        if max_!=None:
            max_[locale__][a] += apper
            max_[locale__][b] += apper
for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_brand_window(brand_window,row,locale__,1)

for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_brand_window(brand_window,row,locale__,1)

if data_type == 'test':
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        locale__ = locale_map[test_locale[i]]
        make_brand_window(brand_window,row,locale__,2)

    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        locale__ = locale_map[test_locale2[i]]
        make_brand_window(brand_window,row,locale__,2)

100%|██████████| 376971/376971 [00:02<00:00, 149676.28it/s]


In [ ]:
for locale_ in ['DE','JP','UK']:
    new_user = []
    brand_ratio = []
    brand_num = []
    brand_item = []
    locale__ = locale_map[locale_]
    for i in tqdm(range(len(train_session))):
        
        if train_locale[i] != locale_:
            continue
        new_user.append(train_user[i])
        row = train_session[i][:-1]
        a = 0
        b = 0
        c = []
        for j in row:
            if brand_map[locale__][j] !=0:
                c.append(j)
        brand_item.append(len(c))
        brand_num.append(len(set(c)))
        brand_ratio.append(len(c)/len(row))
    for i in tqdm(range(len(valid_session))):
        if valid_locale[i] != locale_:
            continue
        new_user.append(valid_user[i])
        row = valid_session[i][:-1]
        a = 0
        b = 0
        c = []
        for j in row:
            if brand_map[locale__][j] !=0:
                c.append(j)
        brand_item.append(len(c))
        brand_num.append(len(set(c)))
        brand_ratio.append(len(c)/len(row))
        
    for i in tqdm(range(len(test_session))):
        if test_locale[i] != locale_:
            continue
        new_user.append(test_user[i])
        row = test_session[i]
        a = 0
        b = 0
        c = []
        for j in row:
            if brand_map[locale__][j] !=0:
                c.append(j)
        brand_item.append(len(c))
        brand_num.append(len(set(c)))
        brand_ratio.append(len(c)/len(row))
    user_brand = pd.DataFrame({'user':new_user,
                             'brand_item':brand_item,
                             'brand_num':brand_num,
                             'brand_ratio':brand_ratio,
                            }).set_index(["user"])
    user_brand.to_parquet('user_brand'+locale_+'.pqt')

100%|██████████| 316972/316972 [00:00<00:00, 652790.44it/s] 


### 11.author

In [ ]:
apper = 1
author = pro_df['author'].tolist()
author = [str(x) for x in author]
author_key = list(set(author))
author_number = len(author_key)
author_values = list(range(1,len(author_key)+1))
author_s2n = dict(zip(author_key, author_values))
author_s2n['nan'] = 0
author_map = [{} for i in range(temp)]
author_locale = pro_df['locale'].tolist()
author_item = pro_df['item'].tolist()

for i in range(len(author)):
    if author_locale[i] not in ['DE','UK','JP']:
        continue
    author_map[locale_map[author_locale[i]]][author_item[i]]= author_s2n[author[i]]

author_window = [[ Counter() for _ in range(author_number+2)] for i in range(temp)]
def make_author_window(cov,row,locale__,apper,max_=None):
    for j in range(len(row)-1):
        a = author_map[locale__][row[j]]
        b = author_map[locale__][row[j+1]]
        if a*b==0:
            continue
        cov[locale__][a][b] += apper
        cov[locale__][b][a] += apper
        if max_!=None:
            max_[locale__][a] += apper
            max_[locale__][b] += apper
    for j in range(len(row)-2):
        a = author_map[locale__][row[j]]
        b = author_map[locale__][row[j+2]]
        if a*b==0:
            continue
        cov[locale__][a][b] += apper
        cov[locale__][b][a] += apper
        if max_!=None:
            max_[locale__][a] += apper
            max_[locale__][b] += apper
for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_author_window(author_window,row,locale__,1)

for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_author_window(author_window,row,locale__,1)
if data_type=='test':
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        locale__ = locale_map[test_locale[i]]
        make_author_window(author_window,row,locale__,2)
    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        locale__ = locale_map[test_locale2[i]]
        make_author_window(author_window,row,locale__,2)

100%|██████████| 376971/376971 [00:01<00:00, 323463.89it/s]


In [ ]:
for locale_ in ['DE','JP','UK']:
    new_user = []
    author_ratio = []
    author_num = []
    author_item = []
    locale__ = locale_map[locale_]
    for i in tqdm(range(len(train_session))):
        
        if train_locale[i] != locale_:
            continue
        new_user.append(train_user[i])
        row = train_session[i][:-1]
        a = 0
        b = 0
        c = []
        for j in row:
            if author_map[locale__][j] !=0:
                c.append(j)
        author_item.append(len(c))
        author_num.append(len(set(c)))
        author_ratio.append(len(c)/len(row))
    for i in tqdm(range(len(valid_session))):
        if valid_locale[i] != locale_:
            continue
        new_user.append(valid_user[i])
        row = valid_session[i][:-1]
        a = 0
        b = 0
        c = []
        for j in row:
            if author_map[locale__][j] !=0:
                c.append(j)
        author_item.append(len(c))
        author_num.append(len(set(c)))
        author_ratio.append(len(c)/len(row))
        
    for i in tqdm(range(len(test_session))):
        if test_locale[i] != locale_:
            continue
        new_user.append(test_user[i])
        row = test_session[i]
        a = 0
        b = 0
        c = []
        for j in row:
            if author_map[locale__][j] !=0:
                c.append(j)
        author_item.append(len(c))
        author_num.append(len(set(c)))
        author_ratio.append(len(c)/len(row))
    user_pop = pd.DataFrame({'user':new_user,
                             'author_item':author_item,
                             'author_num':author_num,
                             'author_ratio':author_ratio,
                            }).set_index(["user"])
    user_pop.to_parquet('user_author'+locale_+'.pqt')

100%|██████████| 316972/316972 [00:00<00:00, 777808.48it/s] 


### 12.material

In [ ]:
def make_material_window(cov,row,locale__,apper,max_=None):
    for j in range(len(row)-1):
        a = material_map[locale__][row[j]]
        b = material_map[locale__][row[j+1]]
        if a*b==0:
            continue
        cov[locale__][a][b] += apper
        cov[locale__][b][a] += apper
        if max_!=None:
            max_[locale__][a] += apper
            max_[locale__][b] += apper
    for j in range(len(row)-2):
        a = material_map[locale__][row[j]]
        b = material_map[locale__][row[j+2]]
        if a*b==0:
            continue
        cov[locale__][a][b] += apper
        cov[locale__][b][a] += apper
        if max_!=None:
            max_[locale__][a] += apper
            max_[locale__][b] += apper

In [ ]:
apper = 1
material = pro_df['material'].tolist()
material = [str(x) for x in material]
material_key = list(set(material))
material_number = len(material_key)
material_values = list(range(1,len(material_key)+1))
material_s2n = dict(zip(material_key, material_values))
material_s2n['nan'] = 0
material_map = [{} for i in range(temp)]
material_locale = pro_df['locale'].tolist()
material_item = pro_df['item'].tolist()


for i in range(len(material)):
    if material_locale[i] not in ['DE','UK','JP']:
        continue
    material_map[locale_map[material_locale[i]]][material_item[i]]= material_s2n[material[i]]

material_window = [[ Counter() for _ in range(material_number+2)] for i in range(temp)]
for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_material_window(material_window,row,locale__,1)
        

for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_material_window(material_window,row,locale__,1)
if data_type == 'test':
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        locale__ = locale_map[test_locale[i]]
        make_material_window(material_window,row,locale__,2)
    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        locale__ = locale_map[test_locale2[i]]
        make_material_window(material_window,row,locale__,2)

100%|██████████| 376971/376971 [00:01<00:00, 237019.96it/s]


In [ ]:
for locale_ in ['DE','JP','UK']:
    new_user = []
    material_ratio = []
    material_num = []
    material_item = []
    locale__ = locale_map[locale_]
    for i in tqdm(range(len(train_session))):
        
        if train_locale[i] != locale_:
            continue
        new_user.append(train_user[i])
        row = train_session[i][:-1]
        a = 0
        b = 0
        c = []
        for j in row:
            if material_map[locale__][j] !=0:
                c.append(j)
        material_item.append(len(c))
        material_num.append(len(set(c)))
        material_ratio.append(len(c)/len(row))
    for i in tqdm(range(len(valid_session))):
        if valid_locale[i] != locale_:
            continue
        new_user.append(valid_user[i])
        row = valid_session[i][:-1]
        a = 0
        b = 0
        c = []
        for j in row:
            if material_map[locale__][j] !=0:
                c.append(j)
        material_item.append(len(c))
        material_num.append(len(set(c)))
        material_ratio.append(len(c)/len(row))
        
    for i in tqdm(range(len(test_session))):
        if test_locale[i] != locale_:
            continue
        new_user.append(test_user[i])
        row = test_session[i]
        a = 0
        b = 0
        c = []
        for j in row:
            if material_map[locale__][j] !=0:
                c.append(j)
        material_item.append(len(c))
        material_num.append(len(set(c)))
        material_ratio.append(len(c)/len(row))
    user_pop = pd.DataFrame({'user':new_user,
                             'material_item':material_item,
                             'material_num':material_num,
                             'material_ratio':material_ratio,
                            }).set_index(["user"])
    user_pop.to_parquet('user_material'+locale_+'.pqt')

100%|██████████| 316972/316972 [00:00<00:00, 675130.07it/s] 


In [ ]:
material_window_max = [[ 0 for _ in range(material_number+1)] for i in range(3)]
brand_window_max = [[ 0 for _ in range(brand_number+1)] for i in range(3)]
author_window_max = [[ 0 for _ in range(author_number+1)] for i in range(3)]
for i in range(3):
    for j in range(1,brand_number+1):
        brand_window_max[i][j] = sum(brand_window[i][j].values())
    for j in range(1,material_number+1):
        material_window_max[i][j] = sum(material_window[i][j].values())
    for j in range(author_number+1):
        author_window_max[i][j] = sum(author_window[i][j].values())

### 14.next item Multi-hop

In [ ]:
def make_item_next(cov,row,locale__,apper,_max=None):
    for j in range(len(row)-1):
        cov[locale__][row[j]][row[j+1]] += apper
        if _max!=None:
            _max[locale__][row[j]] += apper

In [ ]:
item_next_ = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
for i in tqdm(range(len(train_session))):
    row = train_session[i]
    if i % 10 == 0 and data_type == 'train':
        row = row[:-1]
    locale__ = locale_map[train_locale[i]]
    make_item_next(item_next_,row,locale__,1)
    
if data_type == 'test':
    for i in tqdm(range(len(valid_session))):
        row = valid_session[i]
        locale__ = locale_map[valid_locale[i]]
        make_item_next(item_next_,row,locale__,1)
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        locale__ = locale_map[test_locale[i]]
        make_item_next(item_next_,row,locale__,2)
    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        locale__ = locale_map[test_locale2[i]]
        make_item_next(item_next_,row,locale__,2)

100%|██████████| 376971/376971 [00:01<00:00, 274267.78it/s]


In [ ]:
item_next_double = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
for i in range(3):
    for j in tqdm(range(itemnum)):
        for key in item_next_[i][j].keys():
            item_next_double[i][j] += item_next_[i][j]
            sum_ = 0
            for key2 in item_next_[i][key].keys():
                sum_ += item_next_[i][key][key2]
            if sum_ == 0:
                continue
            for key2 in item_next_[i][key].keys():
                if key2 in item_next_[i][j].keys():
                    continue
                item_next_double[i][j][key2] += (item_next_[i][key][key2]/sum_*item_next_[i][key][key2])/20


100%|██████████| 1410676/1410676 [02:49<00:00, 8299.45it/s]   


In [ ]:
item_next_double_max = get_max(item_next_double,itemnum)

### 15.last item Multi-hop

In [ ]:
def make_item_next_last(cov,row,locale__,apper,_max=None): 
    for j in range(len(row)-1):
        cov[locale__][row[j]][row[-1]] += apper
        if _max!=None:
            _max[locale__][row[j]] += apper

In [ ]:
item_next_last_ = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
for i in tqdm(range(len(train_session))):
    if i % 10 == 0 and data_type == 'train':
        continue
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_item_next_last(item_next_last_,row[-2:],locale__,1)
if data_type =='test':
    for i in tqdm(range(len(valid_session))):
        row = valid_session[i]
        locale__ = locale_map[valid_locale[i]]
        make_item_next_last(item_next_last_,row[-2:],locale__,1)
    

100%|██████████| 16364/16364 [00:00<00:00, 411881.92it/s]


In [ ]:
item_next_last_0_2_double = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
for i in range(3):
    for j in tqdm(range(itemnum)):
        for key in item_next_last_[i][j].keys():
            item_next_last_0_2_double[i][j] += item_next_last_[i][j]
            sum_ = 0
            for key2 in item_next_last_[i][key].keys():
                sum_ += item_next_last_[i][key][key2]
            if sum_ == 0:
                continue
            for key2 in item_next_last_[i][key].keys():
                if key2 in item_next_last_[i][j].keys():
                    continue
                item_next_last_0_2_double[i][j][key2] += (item_next_last_[i][key][key2]/sum_*item_next_last_[i][key][key2])/10


100%|██████████| 1410676/1410676 [00:12<00:00, 116553.24it/s] 


In [ ]:
item_next_last_0_2_double_max = get_max(item_next_last_0_2_double,itemnum)

### 18.last back

In [ ]:
def make_item_next_last_back(cov,row,locale__,apper,_max=None): 
    cov[locale__][row[-1]][row[-2]] += apper
    if _max!=None:
        _max[locale__][row[-1]] += apper

In [ ]:
item_next_last_back = [[ Counter() for _ in range(itemnum)] for i in range(temp)]
for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_item_next_last_back(item_next_last_back,row,locale__,1)
for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_item_next_last_back(item_next_last_back,row,locale__,1)
    

100%|██████████| 16364/16364 [00:00<00:00, 482628.69it/s]


In [ ]:
item_next_last_back_max = get_max(item_next_last_back,itemnum)

### 23 item mix

In [ ]:
def make_item_mix(cov,row,locale__,apper,_max=None):
    for j in range(len(row)-1):
        cov[locale__][row[j]][row[j+1]] += apper
        cov[locale__][row[j+1]][row[j]] += apper*0.5
        if _max!=None:
            _max[locale__][row[j]] += apper
            _max[locale__][row[j+1]] += apper*0.5

In [ ]:
item_mix = [[ Counter() for _ in range(itemnum)] for i in range(3)]
for i in tqdm(range(len(train_session))):
    row = train_session[i]
    locale__ = locale_map[train_locale[i]]
    make_item_mix(item_mix,row,locale__,1)
        
for i in tqdm(range(len(valid_session))):
    row = valid_session[i]
    locale__ = locale_map[valid_locale[i]]
    make_item_mix(item_mix,row,locale__,1)
    
if data_type == 'test':
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        locale__ = locale_map[test_locale[i]]
        make_item_mix(item_mix,row,locale__,3)

    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        locale__ = locale_map[test_locale2[i]]
        make_item_mix(item_mix,row,locale__,2)

100%|██████████| 376971/376971 [00:02<00:00, 186277.12it/s]


In [ ]:
item_mix_max = get_max(item_mix,itemnum)

## pop feature

In [ ]:
apper = 1
item_sum_pop = [Counter() for _ in range(len(locale_map))]
if data_type !='test':
    for i in tqdm(range(len(train_session))):
        row = train_session[i]
        if i %10==0 and data_type == 'train':
            row = row[:-1]
        for item in set(row):
            item_sum_pop[locale_map[train_locale[i]]][item]+= apper
if data_type=='test':
    for i in tqdm(range(len(test_session))):
        row = test_session[i]
        for item in set(row):
            item_sum_pop[locale_map[test_locale[i]]][item]= apper
    for i in tqdm(range(len(test_session2))):
        row = test_session2[i]
        for item in set(row):
            item_sum_pop[locale_map[test_locale2[i]]][item]= apper

100%|██████████| 376971/376971 [00:00<00:00, 420447.02it/s]


In [ ]:

from tqdm import tqdm
for locale_ in ['DE','JP','UK']:
    sum_pop_score = []
    item = []
    temp_df = pro_df[pro_df['locale']==locale_]
    temp_item = temp_df['item'].tolist()
    for i in tqdm(range(len(temp_item))):
        item.append(temp_item[i])
        sum_pop_score.append(item_sum_pop[locale_map[locale_]][temp_item[i]])
    pop_df = pd.DataFrame({'item':item,
                           'sum_pop_score':sum_pop_score,
                          }).set_index(["item"])
    pop_df['sum_pop_score'] = pop_df['sum_pop_score'].astype('int32')
    pop_df.to_parquet(data_type+'_item_pop'+locale_+'.pqt')

100%|██████████| 500180/500180 [00:00<00:00, 1224518.17it/s]


## count feature

In [30]:
for locale_ in ['DE','JP','UK']:
    new_user = []
    new_item = []
    for i in tqdm(range(len(train_session))):
        if train_locale[i] != locale_:
            continue
        for item in train_session[i]:
            new_user.append(train_user[i])
            new_item.append(item)
            
    for i in tqdm(range(len(valid_session))):
        if valid_locale[i] != locale_:
            continue
        for item in valid_session[i]:
            new_user.append(valid_user[i])
            new_item.append(item)
    
    for i in tqdm(range(len(test_session))):
        if test_locale[i] != locale_:
            continue
        for item in test_session[i]:
            new_user.append(test_user[i])
            new_item.append(item)
            
    
    df = pd.DataFrame({'user':new_user,'item':new_item}, dtype='int32')
    
    item_features = df.groupby('item').agg({'item':'count','user':'nunique'})
    item_features.columns = ['item_item_count','item_user_count']
    item_features['item_item_count'] = item_features['item_item_count'].astype('int16')
    item_features['item_user_count'] = item_features['item_user_count'].astype('int16')
    # item_features.to_parquet('item_count'+locale_+'.pqt')
    
    new_user = []
    new_item = []
    for i in tqdm(range(len(train_session))):
        if train_locale[i] != locale_:
            continue
        for item in train_session[i][:-1]:
            new_user.append(train_user[i])
            new_item.append(item)
            
    for i in tqdm(range(len(valid_session))):
        if valid_locale[i] != locale_:
            continue
        for item in valid_session[i][:-1]:
            new_user.append(valid_user[i])
            new_item.append(item)
            
    for i in tqdm(range(len(test_session))):
        if test_locale[i] != locale_:
            continue
        for item in test_session[i]:
            new_user.append(test_user[i])
            new_item.append(item)
    df = pd.DataFrame({'user':new_user,'item':new_item}, dtype='int32')
    
    user_features = df.groupby('user').agg({'user':'count','item':'nunique'}, dtype='int32')
    user_features.columns = ['user_user_count','user_item_count']
    user_features['user_user_count'] = user_features['user_user_count'] .astype('int16')
    user_features['user_item_count'] = user_features['user_item_count'] .astype('int16')
    user_features['remember_ratio'] = user_features['user_item_count']/user_features['user_user_count']
    user_features['remember_ratio'] = user_features['remember_ratio'].astype('float32')
    user_features.to_parquet('user_count'+locale_+'.pqt')

100%|██████████| 316971/316971 [00:00<00:00, 1215815.84it/s]


# make feature file

In [ ]:
import gc
def process_train(df, recall, 
                  real_logits=None,
                  name='train', data_type='train',batch=0,back=False,use_apper=False):
    ret_df = []
    sum_len = len(df)
    recall_num = 250
    if True:
        if batch == 1:#iten next
            tcov = item_next
            tcov_max = item_next_max
        elif batch == 2:#item next skip2
            tcov = item_next2
            tcov_max = item_next2_max
        elif batch == 3:#item next skip3
            tcov = item_next3
            tcov_max = item_next3_max
        elif batch == 4:#item back
            tcov = item_next_back
            tcov_max = item_next_back_max
        elif batch == 5:#item back skip2
            tcov = item_next2_back
            tcov_max = item_next2_back_max
        elif batch == 6: #last02
            tcov = item_next_last_0_2
            tcov_max = item_next_last_0_2_max
        elif batch == 7: #last23
            tcov = item_next_last_2_3
            tcov_max = item_next_last_2_3_max
        elif batch == 9: #last3~
            tcov = item_next_last_5_5
            tcov_max = item_next_last_5_5_max
        elif batch == 10:#lasttime
            tcov = item_last_time
            tcov_max = item_last_time_max
        elif batch == 11:#string
            _=1
        elif batch == 12:#cfmodel
            _=2
        elif batch == 13:#unique_pair
            tcov = unique_pair
            tcov_max = unique_pair_max
        elif batch == 15:#brand_window
            tcov = brand_window
            tcov_max = brand_window_max
        elif batch == 16:#author_window
            tcov = author_window
            tcov_max = author_window_max
        elif batch == 17:#material_window
            tcov = material_window
            tcov_max = material_window_max
        elif batch == 18:#brand
            _=1
        elif batch == 19:#author
            _=1
        elif batch == 20:#material
            _=1
        elif batch == 21:#next item Multi-hop
            tcov = item_next_double
            tcov_max = item_next_double_max
        elif batch == 22:#last item Multi-hop
            tcov = item_next_last_0_2_double
            tcov_max = item_next_last_0_2_double_max
        elif batch == 23:#lastback
            tcov = item_next_last_back
            tcov_max = item_next_last_back_max
        elif batch == 27:#item mix
            tcov = item_mix
            tcov_max = item_mix_max
    if batch == 0:
        feat_num = [0,0]
    else:
        if back == False:
            feat_num = [(batch-1)*7+1,(batch)*7]
        else:
            feat_num = [(batch-1)*7+1+700,(batch)*7+700]
    number = 0
    for locale_ in ['DE','JP','UK']:#
        locale__ = locale_map[locale_]
        temp_df = df[df['locale'] == locale_]
        session = temp_df['session'].tolist()
        user = temp_df['user'].tolist()
        index = temp_df.index
        
        if batch == 0:
            new_user = []
            new_item = []
            target = []
            item_pop = pd.read_parquet(data_type+'_item_pop'+locale_+'.pqt')
            user_count = pd.read_parquet('user_count'+locale_+'.pqt')
            user_brand = pd.read_parquet('user_brand'+locale_+'.pqt')
            user_author = pd.read_parquet('user_author'+locale_+'.pqt')
            user_material = pd.read_parquet('user_material'+locale_+'.pqt')
            item_price = pd.read_parquet('item_price'+locale_+'.pqt')
            user_price = pd.read_parquet('user_price'+locale_+'.pqt')
            rank = []
            value = []
            coarse = []
            nn_logits = []
        elif batch == 11:
            string1 = temp_df['string1'].tolist()
            string3 = temp_df['string3'].tolist()
            ui_feat1 = []#-1score
            ui_feat2 = []#-2score
            ui_feat3 = []#-3score
            ui_feat4 = []#5avg score
            ui_feat5 = []#avg score
            ui_feat6 = []#weight score
            ui_feat7 = []#max
        else:
            ui_feat1 = []#-1score
            ui_feat2 = []#-2score
            ui_feat3 = []#-3score
            ui_feat4 = []#5avg score
            ui_feat5 = []#avg score
            ui_feat6 = []#weight score
            ui_feat7 = []#max
        for i in tqdm(range(len(session))):
            if data_type != 'test':
                label = session[i][-1]
            row = session[i]
            if data_type != 'test':
                prev_items = session[i][:-1]
            else:
                prev_items = session[i]
            prev_items = list(dict.fromkeys(prev_items[::-1]))[::-1]
            pad_none = -1
            apper = -1
            if use_apper==True:
                if batch == 1:
                    make_item_next(item_next,row,locale__,apper,item_next_max)
                elif batch == 2:
                    make_item_next2(item_next2,row,locale__,apper,item_next2_max)
                elif batch == 3:
                    make_item_next3(item_next3,row,locale__,apper,item_next3_max)
                elif batch == 4:
                    make_item_next_back(item_next_back,row,locale__,apper,item_next_back_max)
                elif batch == 5:
                    make_item_next2_back(item_next2_back,row,locale__,apper,item_next2_back_max)
                elif batch == 6:
                    make_item_next_last(item_next_last_0_2,row[-2:],locale__,apper,item_next_last_0_2_max)
                elif batch == 7:
                    make_item_next_last(item_next_last_2_3,row[-3:-2] + [row[-1]],locale__,apper,item_next_last_2_3_max)
                elif batch == 9:
                    make_item_next_last(item_next_last_5_5,row[:-3]+ [row[-1]],locale__,apper,item_next_last_5_5_max)
                elif batch == 10:
                    make_item_last_time(item_last_time,row,locale__,apper,item_last_time_max)
                elif batch == 11:
                    check1[string1[i]][label]+=apper
                    check3[string3[i]][label]+=apper
                elif batch == 13:
                    make_unique_pair(unique_pair,row,locale__,apper,unique_pair_max)
                elif batch == 15:
                    make_brand_window(brand_window,row,locale__,apper,brand_window_max)
                elif batch == 16:
                    make_author_window(author_window,row,locale__,apper,author_window_max)
                elif batch == 17:
                    make_material_window(material_window,row,locale__,apper,material_window_max)
                elif batch == 23:
                    make_item_next_last_back(item_next_last_back,row,locale__,apper,item_next_last_back_max)
                elif batch == 27:
                    make_item_mix(item_mix,row,locale__,apper,item_mix_max)
            for j in range(len(recall[index[i]])):
                number += 1
                recall_item = recall[index[i]][j]
                ##############################################featur make#####################################################################
                if batch==0:
                    if data_type != 'test':
                        if recall_item == label:
                            target.append(1)
                        else:
                            target.append(0)
                    new_user.append(user[i])
                    new_item.append(recall_item)
                    nn_logits.append(real_logits[index[i]][j])
                elif batch == 11:
                    if counter1[string1[i]] !=0:
                        ui_feat1.append(check1[string1[i]][recall_item]/max(1,sum(check1[string1[i]].values())))
                    else:
                        ui_feat1.append(0)
                    if counter3[string3[i]] !=0:
                        ui_feat2.append(check3[string3[i]][recall_item]/max(1,sum(check3[string3[i]].values())))
                    else:
                        ui_feat2.append(0)
                    ui_feat3.append(0)
                    ui_feat4.append(0)
                    ui_feat5.append(0)
                    ui_feat6.append(0)
                    ui_feat7.append(0)
                elif batch == 12:
                    ui_feat1.append(item_sim[locale__][prev_items[-1]][recall_item])
                    
                    if len(prev_items)>=2:
                        ui_feat2.append(item_sim[locale__][prev_items[-2]][recall_item])
                    else:
                        ui_feat2.append(pad_none)
                        
                    if len(prev_items)>=3:
                        ui_feat3.append(item_sim[locale__][prev_items[-3]][recall_item])
                    else:
                        ui_feat3.append(pad_none)
                        
                    temp_score = 0.
                    temp_score = 0.
                    for k in range(1,min(6,len(prev_items)+1)):
                        t_score = item_sim[locale__][prev_items[-k]][recall_item]
                        temp_score += sqrt(t_score)
                    ui_feat4.append(temp_score/min(5,len(prev_items)))
                    
                    for k in range(1,len(prev_items)+1):
                        t_score = item_sim[locale__][prev_items[-k]][recall_item]
                        temp_score += sqrt(t_score)
                    ui_feat5.append(temp_score/len(prev_items))
                    
                    temp_score = 0.
                    weight = 1.
                    for k in range(1,len(prev_items)+1):
                        if k >10:
                            break
                        elif k <=1:
                            weight = 1
                        elif k<=2:
                            weight = 1/20
                        elif k<=3:
                            weight = 1/400
                        elif k<=5:
                            weight = 1/8000
                        t_score = item_sim[locale__][prev_items[-k]][recall_item]
                        temp_score += sqrt(t_score)*weight
                    ui_feat6.append(temp_score)
                    
                    max_score = 0
                    for k in range(1,len(prev_items)+1):
                        max_score = max(max_score,item_sim[locale__][prev_items[-k]][recall_item])
                    ui_feat7.append(max_score)
                elif batch == 15:
                    a = brand_map[locale__][prev_items[-1]]
                    b = brand_map[locale__][recall_item]
                    ui_feat1.append(sqrt(tcov[locale__][a][b]/max(1,tcov_max[locale__][a])))
                    if len(prev_items)>=2:
                        a = brand_map[locale__][prev_items[-2]]
                        ui_feat2.append(sqrt(tcov[locale__][a][b]/max(1,tcov_max[locale__][a])))
                    else:
                        ui_feat2.append(pad_none)
                    if len(prev_items)>=3:
                        a = brand_map[locale__][prev_items[-3]]
                        ui_feat3.append(sqrt(tcov[locale__][a][b]/max(1,tcov_max[locale__][a])))
                    else:
                        ui_feat3.append(pad_none)
                    temp_score = 0.
                    for k in range(1,min(6,len(prev_items)+1)):
                        a = brand_map[locale__][prev_items[-k]]
                        t_score = tcov[locale__][a][b]/max(1,tcov_max[locale__][a])
                        temp_score += sqrt(t_score)
                    ui_feat4.append(temp_score/min(5,len(prev_items)))
                    
                    for k in range(1,len(prev_items)+1):
                        a = brand_map[locale__][prev_items[-k]]
                        t_score = tcov[locale__][a][b]/max(1,tcov_max[locale__][a])
                        temp_score += sqrt(t_score)
                    ui_feat5.append(temp_score/len(prev_items))
                    
                    temp_score = 0.
                    weight = 1.
                    for k in range(1,len(prev_items)+1):
                        if k >10:
                            break
                        elif k <=1:
                            weight = 1
                        elif k<=2:
                            weight = 1/20
                        elif k<=3:
                            weight = 1/400
                        elif k<=5:
                            weight = 1/8000
                        a = brand_map[locale__][prev_items[-k]]
                        t_score = tcov[locale__][a][b]/max(1,tcov_max[locale__][a])
                        temp_score += sqrt(t_score)*weight
                    ui_feat6.append(temp_score)
                    
                    max_score = 0
                    for k in range(1,len(prev_items)+1):
                        a = brand_map[locale__][prev_items[-k]]
                        max_score = max(max_score,tcov[locale__][a][b]/max(1,tcov_max[locale__][a]))
                    ui_feat7.append(max_score)
                elif batch == 16:
                    a = author_map[locale__][prev_items[-1]]
                    b = author_map[locale__][recall_item]
                    ui_feat1.append(sqrt(tcov[locale__][a][b]/max(1,tcov_max[locale__][a])))
                    if len(prev_items)>=2:
                        a = author_map[locale__][prev_items[-2]]
                        ui_feat2.append(sqrt(tcov[locale__][a][b]/max(1,tcov_max[locale__][a])))
                    else:
                        ui_feat2.append(pad_none)
                    if len(prev_items)>=3:
                        a = author_map[locale__][prev_items[-3]]
                        ui_feat3.append(sqrt(tcov[locale__][a][b]/max(1,tcov_max[locale__][a])))
                    else:
                        ui_feat3.append(pad_none)
                    temp_score = 0.
                    for k in range(1,min(6,len(prev_items)+1)):
                        a = author_map[locale__][prev_items[-k]]
                        t_score = tcov[locale__][a][b]/max(1,tcov_max[locale__][a])
                        temp_score += sqrt(t_score)
                    ui_feat4.append(temp_score/min(5,len(prev_items)))
                    
                    for k in range(1,len(prev_items)+1):
                        a = author_map[locale__][prev_items[-k]]
                        t_score = tcov[locale__][a][b]/max(1,tcov_max[locale__][a])
                        temp_score += sqrt(t_score)
                    ui_feat5.append(temp_score/len(prev_items))
                    
                    temp_score = 0.
                    weight = 1.
                    for k in range(1,len(prev_items)+1):
                        if k >10:
                            break
                        elif k <=1:
                            weight = 1
                        elif k<=2:
                            weight = 1/20
                        elif k<=3:
                            weight = 1/400
                        elif k<=5:
                            weight = 1/8000
                        a = author_map[locale__][prev_items[-k]]
                        t_score = tcov[locale__][a][b]/max(1,tcov_max[locale__][a])
                        temp_score += sqrt(t_score)*weight
                    ui_feat6.append(temp_score)
                    
                    max_score = 0
                    for k in range(1,len(prev_items)+1):
                        a = author_map[locale__][prev_items[-k]]
                        max_score = max(max_score,tcov[locale__][a][b]/max(1,tcov_max[locale__][a]))
                    ui_feat7.append(max_score)
                elif batch == 17:
                    a = material_map[locale__][prev_items[-1]]
                    b = material_map[locale__][recall_item]
                    ui_feat1.append(sqrt(tcov[locale__][a][b]/max(1,tcov_max[locale__][a])))
                    if len(prev_items)>=2:
                        a = material_map[locale__][prev_items[-2]]
                        ui_feat2.append(sqrt(tcov[locale__][a][b]/max(1,tcov_max[locale__][a])))
                    else:
                        ui_feat2.append(pad_none)
                    if len(prev_items)>=3:
                        a = material_map[locale__][prev_items[-3]]
                        ui_feat3.append(sqrt(tcov[locale__][a][b]/max(1,tcov_max[locale__][a])))
                    else:
                        ui_feat3.append(pad_none)
                    temp_score = 0.
                    for k in range(1,min(6,len(prev_items)+1)):
                        a = material_map[locale__][prev_items[-k]]
                        t_score = tcov[locale__][a][b]/max(1,tcov_max[locale__][a])
                        temp_score += sqrt(t_score)
                    ui_feat4.append(temp_score/min(5,len(prev_items)))
                    
                    for k in range(1,len(prev_items)+1):
                        a = material_map[locale__][prev_items[-k]]
                        t_score = tcov[locale__][a][b]/max(1,tcov_max[locale__][a])
                        temp_score += sqrt(t_score)
                    ui_feat5.append(temp_score/len(prev_items))
                    
                    temp_score = 0.
                    weight = 1.
                    for k in range(1,len(prev_items)+1):
                        if k >10:
                            break
                        elif k <=1:
                            weight = 1
                        elif k<=2:
                            weight = 1/20
                        elif k<=3:
                            weight = 1/400
                        elif k<=5:
                            weight = 1/8000
                        a = material_map[locale__][prev_items[-k]]
                        t_score = tcov[locale__][a][b]/max(1,tcov_max[locale__][a])
                        temp_score += sqrt(t_score)*weight
                    ui_feat6.append(temp_score)
                    
                    max_score = 0
                    for k in range(1,len(prev_items)+1):
                        a = material_map[locale__][prev_items[-k]]
                        max_score = max(max_score,tcov[locale__][a][b]/max(1,tcov_max[locale__][a]))
                    ui_feat7.append(max_score)
                elif batch == 18:
                    if brand_map[locale__][recall_item]==0:
                        ui_feat1.append(-1)
                        ui_feat2.append(-1)
                        ui_feat3.append(-1)
                        ui_feat4.append(-1)
                        ui_feat5.append(-1)
                        ui_feat6.append(-1)
                        ui_feat7.append(-1)
                    else:
                        temp_score = 0.
                        for k in range(1,len(prev_items)+1):
                            if brand_map[locale__][recall_item] == brand_map[locale__][prev_items[-k]] and brand_map[locale__][recall_item]*brand_map[locale__][prev_items[-k]]!=0:
                                temp_score+=1
                        ui_feat1.append(sqrt(temp_score/len(prev_items)))
                        if brand_map[locale__][recall_item] == brand_map[locale__][prev_items[-1]] and brand_map[locale__][recall_item]*brand_map[locale__][prev_items[-1]]!=0:
                            ui_feat2.append(1)
                        else:
                            ui_feat2.append(0)

                        temp_score = 0.
                        for k in range(1,min(4,len(prev_items)+1)):
                            if brand_map[locale__][recall_item] == brand_map[locale__][prev_items[-k]] and brand_map[locale__][recall_item]*brand_map[locale__][prev_items[-k]]!=0:
                                temp_score+=1
                        ui_feat3.append(sqrt(temp_score/min(3,len(prev_items))))
                        temp_score = 0.
                        for k in range(1,min(6,len(prev_items)+1)):
                            if brand_map[locale__][recall_item] == brand_map[locale__][prev_items[-k]] and brand_map[locale__][recall_item]*brand_map[locale__][prev_items[-k]]!=0:
                                temp_score+=1
                        ui_feat4.append(sqrt(temp_score/min(5,len(prev_items))))
                        temp_score = 0.
                        for k in range(1,len(prev_items)+1):
                            if brand_map[locale__][recall_item] == brand_map[locale__][prev_items[-k]] and brand_map[locale__][recall_item]*brand_map[locale__][prev_items[-k]]!=0:
                                temp_score+=1
                        ui_feat5.append(sqrt(temp_score/len(prev_items)))
                        temp_score = 0.
                        weight = 1.
                        for k in range(1,len(prev_items)+1):
                            if k >10:
                                break
                            elif k <=1:
                                weight = 1
                            elif k<=2:
                                weight = 1/20
                            elif k<=3:
                                weight = 1/400
                            elif k<=5:
                                weight = 1/8000
                            if brand_map[locale__][recall_item] == brand_map[locale__][prev_items[-k]] and brand_map[locale__][recall_item]*brand_map[locale__][prev_items[-k]]!=0:
                                t_score = 1
                            else:
                                t_score = 0
                            temp_score += t_score*weight
                        ui_feat6.append(temp_score)
                        ui_feat7.append(-1)
                elif batch == 19:
                    if author_map[locale__][recall_item]==0:
                        ui_feat1.append(-1)
                        ui_feat2.append(-1)
                        ui_feat3.append(-1)
                        ui_feat4.append(-1)
                        ui_feat5.append(-1)
                        ui_feat6.append(-1)
                        ui_feat7.append(-1)
                    else:
                        temp_score = 0.
                        for k in range(1,len(prev_items)+1):
                            if author_map[locale__][recall_item] == author_map[locale__][prev_items[-k]] and author_map[locale__][recall_item]*author_map[locale__][prev_items[-k]]!=0:
                                temp_score+=1
                        ui_feat1.append(sqrt(temp_score/len(prev_items)))
                        if author_map[locale__][recall_item] == author_map[locale__][prev_items[-1]] and author_map[locale__][recall_item]*author_map[locale__][prev_items[-1]]!=0:
                            ui_feat2.append(1)
                        else:
                            ui_feat2.append(0)

                        temp_score = 0.
                        for k in range(1,min(4,len(prev_items)+1)):
                            if author_map[locale__][recall_item] == author_map[locale__][prev_items[-k]] and author_map[locale__][recall_item]*author_map[locale__][prev_items[-k]]!=0:
                                temp_score+=1
                        ui_feat3.append(sqrt(temp_score/min(3,len(prev_items))))
                        temp_score = 0.
                        for k in range(1,min(6,len(prev_items)+1)):
                            if author_map[locale__][recall_item] == author_map[locale__][prev_items[-k]] and author_map[locale__][recall_item]*author_map[locale__][prev_items[-k]]!=0:
                                temp_score+=1
                        ui_feat4.append(sqrt(temp_score/min(5,len(prev_items))))
                        temp_score = 0.
                        for k in range(1,len(prev_items)+1):
                            if author_map[locale__][recall_item] == author_map[locale__][prev_items[-k]] and author_map[locale__][recall_item]*author_map[locale__][prev_items[-k]]!=0:
                                temp_score+=1
                        ui_feat5.append(sqrt(temp_score/len(prev_items)))
                        temp_score = 0.
                        weight = 1.
                        for k in range(1,len(prev_items)+1):
                            if k >10:
                                break
                            elif k <=1:
                                weight = 1
                            elif k<=2:
                                weight = 1/20
                            elif k<=3:
                                weight = 1/400
                            elif k<=5:
                                weight = 1/8000
                            if author_map[locale__][recall_item] == author_map[locale__][prev_items[-k]] and author_map[locale__][recall_item]*author_map[locale__][prev_items[-k]]!=0:
                                t_score = 1
                            else:
                                t_score = 0
                            temp_score += t_score*weight
                        ui_feat6.append(temp_score)
                        ui_feat7.append(temp_score)
                elif batch == 20:
                    if material_map[locale__][recall_item]==0:
                        ui_feat1.append(-1)
                        ui_feat2.append(-1)
                        ui_feat3.append(-1)
                        ui_feat4.append(-1)
                        ui_feat5.append(-1)
                        ui_feat6.append(-1)
                        ui_feat7.append(-1)
                    else:
                        temp_score = 0.
                        for k in range(1,len(prev_items)+1):
                            if material_map[locale__][recall_item] == material_map[locale__][prev_items[-k]] and material_map[locale__][recall_item]*material_map[locale__][prev_items[-k]]!=0:
                                temp_score+=1
                        ui_feat1.append(sqrt(temp_score/len(prev_items)))
                        if material_map[locale__][recall_item] == material_map[locale__][prev_items[-1]] and material_map[locale__][recall_item]*material_map[locale__][prev_items[-1]]!=0:
                            ui_feat2.append(1)
                        else:
                            ui_feat2.append(0)

                        temp_score = 0.
                        for k in range(1,min(4,len(prev_items)+1)):
                            if material_map[locale__][recall_item] == material_map[locale__][prev_items[-k]] and material_map[locale__][recall_item]*material_map[locale__][prev_items[-k]]!=0:
                                temp_score+=1
                        ui_feat3.append(sqrt(temp_score/min(3,len(prev_items))))
                        temp_score = 0.
                        for k in range(1,min(6,len(prev_items)+1)):
                            if material_map[locale__][recall_item] == material_map[locale__][prev_items[-k]] and material_map[locale__][recall_item]*material_map[locale__][prev_items[-k]]!=0:
                                temp_score+=1
                        ui_feat4.append(sqrt(temp_score/min(5,len(prev_items))))
                        temp_score = 0.
                        for k in range(1,len(prev_items)+1):
                            if material_map[locale__][recall_item] == material_map[locale__][prev_items[-k]] and material_map[locale__][recall_item]*material_map[locale__][prev_items[-k]]!=0:
                                temp_score+=1
                        ui_feat5.append(sqrt(temp_score/len(prev_items)))
                        temp_score = 0.
                        weight = 1.
                        for k in range(1,len(prev_items)+1):
                            if k >10:
                                break
                            elif k <=1:
                                weight = 1
                            elif k<=2:
                                weight = 1/20
                            elif k<=3:
                                weight = 1/400
                            elif k<=5:
                                weight = 1/8000
                            if material_map[locale__][recall_item] == material_map[locale__][prev_items[-k]] and material_map[locale__][recall_item]*material_map[locale__][prev_items[-k]]!=0:
                                t_score = 1
                            else:
                                t_score = 0
                            temp_score += t_score*weight
                        ui_feat6.append(temp_score)
                        ui_feat7.append(-1)
                elif batch == 50:
                    x = prev_items[-1]
                    y = recall_item
                    score = 0
                    user_set = set(item_user[locale__][x]) & set(item_user[locale__][y])
                    for ua in user_set:
                        for ub in user_set:
                            if ua == ub:
                                continue
                            ua_session = session_all[ua]
                            ub_session = session_all[ub]
                            score += 1/np.sqrt(len(set(ua_session))*len(set(ub_session)))/(1+len(set(ua_session)&set(ub_session)))
                    ui_feat1.append(score)
                    ui_feat2.append(0)
                    ui_feat3.append(0)
                    ui_feat4.append(0)
                    ui_feat5.append(0)
                    ui_feat6.append(0)
                    ui_feat7.append(0)
                else:
                    if back == False:
                        #普通1
                        min_sum = 1
                        ui_feat1.append(sqrt(max(0,tcov[locale__][prev_items[-1]][recall_item])/max(min_sum,tcov_max[locale__][prev_items[-1]])))
                        if len(prev_items)>=2:
                            ui_feat2.append(sqrt(max(0,tcov[locale__][prev_items[-2]][recall_item])/max(min_sum,tcov_max[locale__][prev_items[-2]])))
                        else:
                            ui_feat2.append(pad_none)
                        if len(prev_items)>=3:
                            ui_feat3.append(sqrt(max(0,tcov[locale__][prev_items[-3]][recall_item])/max(min_sum,tcov_max[locale__][prev_items[-3]])))
                        else:
                            ui_feat3.append(pad_none)
                        temp_score = 0.
                        for k in range(1,min(6,len(prev_items)+1)):
                            t_score = max(0,tcov[locale__][prev_items[-k]][recall_item])/max(min_sum,tcov_max[locale__][prev_items[-k]])
                            temp_score += sqrt(t_score)
                        ui_feat4.append(temp_score/min(5,len(prev_items)))
                        temp_score = 0.
                        for k in range(1,len(prev_items)+1):
                            t_score = max(0,tcov[locale__][prev_items[-k]][recall_item])/max(min_sum,tcov_max[locale__][prev_items[-k]])
                            temp_score += sqrt(t_score)
                        ui_feat5.append(temp_score/len(prev_items))
                        temp_score = 0.
                        weight = 1.
                        for k in range(1,len(prev_items)+1):
                            if k >10:
                                break
                            elif k <=1:
                                weight = 1
                            elif k<=2:
                                weight = 1/20
                            elif k<=3:
                                weight = 1/400
                            elif k<=5:
                                weight = 1/8000
                            t_score = max(0,tcov[locale__][prev_items[-k]][recall_item])/max(min_sum,tcov_max[locale__][prev_items[-k]])
                            temp_score += sqrt(t_score)*weight
                        ui_feat6.append(temp_score)
                        max_score = 0
                        for k in range(1,len(prev_items)+1):
                            max_score = max(max_score,max(0,tcov[locale__][prev_items[-k]][recall_item])/max(min_sum,tcov_max[locale__][prev_items[-k]]))
                        ui_feat7.append(max_score)
                    else:
                        #普通1
                        ui_feat1.append(sqrt(max(0,tcov[locale__][recall_item][prev_items[-1]])/max(min_sum,tcov_max[locale__][recall_item])))
                        if len(prev_items)>=2:
                            ui_feat2.append(sqrt(max(0,tcov[locale__][recall_item][prev_items[-2]])/max(min_sum,tcov_max[locale__][recall_item])))
                        else:
                            ui_feat2.append(pad_none)
                        if len(prev_items)>=3:
                            ui_feat3.append(sqrt(max(0,tcov[locale__][recall_item][prev_items[-3]])/max(min_sum,tcov_max[locale__][recall_item])))
                        else:
                            ui_feat3.append(pad_none)
                        temp_score = 0.
                        for k in range(1,min(6,len(prev_items)+1)):
                            t_score = max(0,tcov[locale__][recall_item][prev_items[-k]])/max(min_sum,tcov_max[locale__][recall_item])
                            temp_score += sqrt(t_score)
                        ui_feat4.append(temp_score/min(5,len(prev_items)))
                        temp_score = 0.
                        for k in range(1,len(prev_items)+1):
                            t_score = max(0,tcov[locale__][recall_item][prev_items[-k]])/max(min_sum,tcov_max[locale__][recall_item])
                            temp_score += sqrt(t_score)
                        ui_feat5.append(temp_score/len(prev_items))
                        temp_score = 0.
                        weight = 1.
                        for k in range(1,len(prev_items)+1):
                            if k >10:
                                break
                            elif k <=1:
                                weight = 1
                            elif k<=2:
                                weight = 1/20
                            elif k<=3:
                                weight = 1/400
                            elif k<=5:
                                weight = 1/8000
                            t_score = max(0,tcov[locale__][recall_item][prev_items[-k]])/max(min_sum,tcov_max[locale__][recall_item])
                            temp_score += sqrt(t_score)*weight
                        ui_feat6.append(temp_score)
                        max_score = 0
                        for k in range(1,len(prev_items)+1):
                            max_score = max(max_score,max(0,tcov[locale__][recall_item][prev_items[-k]])/max(min_sum,tcov_max[locale__][recall_item]))
                        ui_feat7.append(max_score)
                ##################################################################################################################        
            apper = 1
            if use_apper==True:
                if batch == 1:
                    make_item_next(item_next,row,locale__,apper,item_next_max)
                elif batch == 2:
                    make_item_next2(item_next2,row,locale__,apper,item_next2_max)
                elif batch == 3:
                    make_item_next3(item_next3,row,locale__,apper,item_next3_max)
                elif batch == 4:
                    make_item_next_back(item_next_back,row,locale__,apper,item_next_back_max)
                elif batch == 5:
                    make_item_next2_back(item_next2_back,row,locale__,apper,item_next2_back_max)
                elif batch == 6:
                    make_item_next_last(item_next_last_0_2,row[-2:],locale__,apper,item_next_last_0_2_max)
                elif batch == 7:
                    make_item_next_last(item_next_last_2_3,row[-3:-2] + [row[-1]],locale__,apper,item_next_last_2_3_max)
                elif batch == 9:
                    make_item_next_last(item_next_last_5_5,row[:-3]+ [row[-1]],locale__,apper,item_next_last_5_5_max)
                elif batch == 10:
                    make_item_last_time(item_last_time,row,locale__,apper,item_last_time_max)
                elif batch == 11:
                    check1[string1[i]][label]+=apper
                    check3[string3[i]][label]+=apper
                elif batch == 13:
                    make_unique_pair(unique_pair,row,locale__,apper,unique_pair_max)
                elif batch == 15:
                    make_brand_window(brand_window,row,locale__,apper,brand_window_max)
                elif batch == 16:
                    make_author_window(author_window,row,locale__,apper,author_window_max)
                elif batch == 17:
                    make_material_window(material_window,row,locale__,apper,material_window_max)
                elif batch == 23:
                    make_item_next_last_back(item_next_last_back,row,locale__,apper,item_next_last_back_max)
                elif batch == 27:
                    make_item_mix(item_mix,row,locale__,apper,item_mix_max)
            
        if number!=0:
            number = 0
            output = pd.DataFrame()
            if batch == 0:
                output = pd.DataFrame({'user':new_user,'item':new_item}, dtype='int32')
                output = output.merge(user_count, left_on='user', right_index=True, how='left').fillna(-1)
                output = output.merge(item_pop, left_on='item', right_index=True, how='left').fillna(-1)
                output = output.merge(item_price, left_on='item', right_index=True, how='left').fillna(-1)
                output = output.merge(user_price, left_on='user', right_index=True, how='left').fillna(-1)
                output = output.merge(user_brand, left_on='user', right_index=True, how='left').fillna(-1)
                output = output.merge(user_author, left_on='user', right_index=True, how='left').fillna(-1)
                output = output.merge(user_material, left_on='user', right_index=True, how='left').fillna(-1)
                output['nn_logits'] = pd.DataFrame(nn_logits, dtype='float32')
                output['pop'] = np.sqrt(output['sum_pop_score']/max(output['sum_pop_score']))
                if data_type != 'test':
                    output['target'] = pd.DataFrame(target, dtype='int8')
            else:
                output['ui_feat'+str(0+feat_num[0])] = pd.DataFrame(ui_feat1,dtype='float32')
                output['ui_feat'+str(1+feat_num[0])] = pd.DataFrame(ui_feat2,dtype='float32')
                output['ui_feat'+str(2+feat_num[0])] = pd.DataFrame(ui_feat3,dtype='float32')
                output['ui_feat'+str(3+feat_num[0])] = pd.DataFrame(ui_feat4,dtype='float32')
                output['ui_feat'+str(4+feat_num[0])] = pd.DataFrame(ui_feat5,dtype='float32')
                output['ui_feat'+str(5+feat_num[0])] = pd.DataFrame(ui_feat6,dtype='float32')
                output['ui_feat'+str(6+feat_num[0])] = pd.DataFrame(ui_feat7,dtype='float32')
            ret_df.append(output)
            ret_df = pd.concat(ret_df).reset_index(drop=True)
            ret_df.to_parquet(name+'_data_'+locale_+'_'+str(feat_num[0])+'~'+str(feat_num[1])+'.pqt')
            ret_df = []
            # return
            

In [ ]:
if data_type == 'train':
    train_recall_data =  pd.read_pickle('./recall_train_250_with_nn_p2.dataset')
    # new_logits = pd.read_pickle('./logits_train_2.dataset')
    for i in [0,1,2,3,4,5,6,7,9,10,11,12,13,16,15,17,18,19,21,22,23,27]:
        process_train(train_data.iloc[list(range(0,len(train_data),10))].reset_index(drop=True),
                    train_recall_data['next_item_prediction'].tolist(),
                    train_recall_data['nn_logits'].tolist(),
                    data_type='train',
                    name='train5',
                    batch=i,#This parameter indicates which matrix to use to build the feature.The numbers here are not the numbers in markdown when the matrix was built above.
                    use_apper=True,
                    # back=True,
                    )
    del train_recall_data

In [ ]:
if data_type == 'train':
    valid_recall_data =  pd.read_pickle('./recall_valid_250_with_nn_p2.dataset')
    # new_logits = pd.read_pickle('./logits_valid_2.dataset')
    for i in [0,1,2,3,4,5,6,7,9,10,11,12,13,16,15,17,18,19,21,22,23,27]:
        process_train(valid_data,
                    valid_recall_data['next_item_prediction'].tolist(),
                    valid_recall_data['nn_logits'].tolist(),
                    data_type='train',
                    name='valid5',
                    batch=i,
                    use_apper=True,
                    # back=True,
                    )

In [ ]:
if data_type == 'test':
    test_recall_data =  pd.read_pickle('./recall_test_250_with_nn_p2.dataset')
    # new_logits = pd.read_pickle('./logits_test_2.dataset')
    for i in [0,1,2,3,4,5,6,7,9,10,11,12,13,16,15,17,18,19,21,22,23,27]:
        process_train(test_data,
                    test_recall_data['next_item_prediction'].tolist(),
                    test_recall_data['nn_logits'].tolist(),
                    data_type='test',
                    name='test5',
                    batch=i,
                    use_apper=False,
                    )
    del test_recall_data

# text feature

In [11]:
import torch
import os
import torch.nn as nn
from torch.cuda.amp import autocast
import torch.nn.functional as F
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
itemnum = 1410675 + 1
#locale map number
locale_map = {}
temp = 0
for item in list(set(train_locale)):
    locale_map[item] = temp
    temp += 1
id2index = [{} for _ in range(temp)]
item = pro_df['item'].tolist()
item_locale = pro_df['locale'].tolist()
for i in range(len(item)):
    if item_locale[i] not in ['DE','UK','JP']:
        continue
    locale_ = locale_map[item_locale[i]]
    id2index[locale_][item[i]] = i

## train

In [23]:
import gc
def process_train(df, recall, text_embedding, name='train',batch=0):
    ret_df = []
    sum_len = len(df)
    recall_num = 250
    apper = 1
    feat_num = [200,206]
    number = 0
    for locale_ in ['DE','JP','UK']:#
        locale__ = locale_map[locale_]
        temp_df = df[df['locale'] == locale_]
        session = temp_df['session'].tolist()
        user = temp_df['user'].tolist()
        index = temp_df.index
        ui_feat1 = []#-1 sim
        ui_feat2 = []#-2 sim
        ui_feat3 = []#-3 sim
        ui_feat4 = []#-5 avg sim
        ui_feat5 = []#avg sim
        ui_feat6 = []#weight sim
        ui_feat7 = []#max

        for i in tqdm(range(len(session))):
            row = session[i]
            temp_list = [1,2,3,5,20]
            prev_items = session[i]
            if name[:4] != 'test':
                prev_items = prev_items[:-1]
            prev_items = list(dict.fromkeys(prev_items[::-1]))
            with autocast():
                prev_embedding = text_embedding[torch.tensor([id2index[locale__][x] for x in prev_items]).to(device),:]
                recall_embedding = text_embedding[torch.tensor([id2index[locale__][x] for x in recall[index[i]]]).to(device),:]
                sim = recall_embedding.mm(prev_embedding.T).cpu()
            
            for j in range(sim.shape[0]):
                number += 1
                
                ##############################################featur make#####################################################################
                ui_feat1.append(sim[j][0].cpu().item())
                if sim.shape[1]>=2:
                    ui_feat2.append(sim[j][1].cpu().item())
                else:
                    ui_feat2.append(0)
                if sim.shape[1]>=3:
                    ui_feat3.append(sim[j][2].cpu().item())
                else:
                    ui_feat3.append(0)
                ui_feat4.append(sim[j][:5].mean(dim=0).cpu().item())
                ui_feat5.append(sim[j].mean(dim=0).cpu().item())
                temp_score = 0.
                weight = 1.
                for k in range(0,sim.shape[1]):
                    if k >10:
                        break
                    elif k <1:
                        weight = 1
                    elif k<2:
                        weight = 1/20
                    elif k<3:
                        weight = 1/400
                    elif k<5:
                        weight = 1/8000
                    temp_score += sim[j][k].cpu().item()*weight
                ui_feat6.append(temp_score)
                ui_feat7.append(sim[j].max(0).values.item())
                ##################################################################################################################        

        output = pd.DataFrame()
        output['ui_feat200'] = pd.DataFrame(ui_feat1,dtype='float32')
        output['ui_feat201'] = pd.DataFrame(ui_feat2,dtype='float32')
        output['ui_feat202'] = pd.DataFrame(ui_feat3,dtype='float32')
        output['ui_feat203'] = pd.DataFrame(ui_feat4,dtype='float32')
        output['ui_feat204'] = pd.DataFrame(ui_feat5,dtype='float32')
        output['ui_feat205'] = pd.DataFrame(ui_feat6,dtype='float32')
        output['ui_feat206'] = pd.DataFrame(ui_feat7,dtype='float32')
        output.to_parquet(name+'_data_'+locale_+'_'+str(feat_num[0])+'~'+str(feat_num[1])+'.pqt')

In [ ]:
title_embedding = torch.load('../data/title_embeddings_128.dataset').to(device)
title_embedding = F.normalize(title_embedding, p=2, dim=1)
train_recall_data =  pd.read_pickle('./recall_train_250_with_nn.dataset')['next_item_prediction'].tolist()
process_train(train_data.iloc[list(range(0,len(train_data),10))].reset_index(drop=True),
          train_recall_data,
          title_embedding,
          batch=0,
            name='train5'
         )

 97%|█████████▋| 114456/117970 [29:52<00:57, 61.20it/s]

## valid&test

In [ ]:
del train_recall_data

In [21]:
valid_recall_data =  pd.read_pickle('./recall_valid_250_with_nn.dataset')['next_item_prediction'].tolist()
process_train(valid_data,
              valid_recall_data,
              title_embedding.to(device),
              batch=0,
              name='valid5'
             )

100%|██████████| 5838/5838 [01:31<00:00, 63.81it/s]


In [24]:
test_recall_data =  pd.read_pickle('./recall_test_250_with_nn.dataset')['next_item_prediction'].tolist()
process_train(test_data,
              test_recall_data,
              title_embedding.to(device),
              batch=0,
              name='test5'
             )

100%|██████████| 115936/115936 [39:58<00:00, 48.34it/s]
